In [1]:
import cupy as cp
import cusignal 
from scipy import signal
import numpy as np

### Resample

In [2]:
start = 0
stop = 10
num = int(1e8)
resample_num = int(1e5)

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [3]:
%%time
cf = signal.resample(cy, resample_num, window=('kaiser', 0.5))

CPU times: user 13.7 s, sys: 3.87 s, total: 17.5 s
Wall time: 17.5 s


In [4]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [5]:
%%time
gf = cusignal.resample(gy, resample_num, window=('kaiser',0.5))

CPU times: user 195 ms, sys: 114 ms, total: 310 ms
Wall time: 321 ms


### Resample Poly

In [2]:
start = 0
stop = 10
num = int(1e8)
resample_up = 2
resample_down = 3

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [3]:
%%time
cf = signal.resample_poly(cy, resample_up, resample_down, window=('kaiser', 0.5))

CPU times: user 2.32 s, sys: 288 ms, total: 2.61 s
Wall time: 2.61 s


In [4]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [5]:
%%time
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=True)

CPU times: user 252 ms, sys: 44.3 ms, total: 297 ms
Wall time: 295 ms


In [6]:
%%time
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=False)

TypeError: resample_poly() got an unexpected keyword argument 'use_numba'

In [10]:
gpu_signal = cusignal.get_shared_mem(num, dtype=np.complex128)
gpu_signal[:] = cy

### FIR Filter Design with Window

In [13]:
numtaps = int(1e8)
f1, f2 = 0.1, 0.2

In [14]:
%%time
cfirwin = signal.firwin(numtaps, [f1, f2], pass_zero=False)

CPU times: user 11 s, sys: 4.98 s, total: 16 s
Wall time: 16 s


In [15]:
%%time
gfirwin = cusignal.firwin(numtaps, [f1, f2], pass_zero=False)

CPU times: user 214 ms, sys: 20.8 ms, total: 235 ms
Wall time: 233 ms


### Correlate

In [16]:
sig = np.random.rand(int(1e8))
sig_noise = sig + np.random.randn(len(sig))

In [17]:
%%time
ccorr = signal.correlate(sig_noise, np.ones(128), mode='same') / 1e6

CPU times: user 15 s, sys: 5.1 s, total: 20.1 s
Wall time: 20 s


In [18]:
sig = cp.random.rand(int(1e8))
sig_noise = sig + cp.random.randn(len(sig))

In [19]:
%%time
gcorr = cusignal.correlate(sig_noise, cp.ones(128), mode='same') / 1e6

CPU times: user 74.2 ms, sys: 12.8 ms, total: 87 ms
Wall time: 85.2 ms


### Convolve

In [20]:
sig = np.random.rand(int(1e8))
win = signal.windows.hann(int(1e3))

In [21]:
%%time
cconv = signal.convolve(sig, win, mode='same') / np.sum(sig)

CPU times: user 14.9 s, sys: 5.29 s, total: 20.2 s
Wall time: 20.2 s


In [22]:
%%time
sig = cp.random.rand(int(1e8))
win = cusignal.hann(int(1e3))
gconv = cusignal.convolve(sig, win, mode='same') / cp.sum(win)

CPU times: user 60.4 ms, sys: 35.3 ms, total: 95.7 ms
Wall time: 93.8 ms


### Convolution using the FFT Method

In [23]:
csig = np.random.randn(int(1e8))

In [24]:
%%time
cautocorr = signal.fftconvolve(csig, csig[::-1], mode='full')

CPU times: user 24 s, sys: 9.43 s, total: 33.4 s
Wall time: 33.4 s


In [25]:
gsig = cp.random.randn(int(1e8))

In [26]:
%%time
gautocorr = cusignal.fftconvolve(gsig, gsig[::-1], mode='full')

CPU times: user 101 ms, sys: 28.4 ms, total: 129 ms
Wall time: 132 ms


### Wiener Filter on N-Dimensional Array

In [27]:
csig = np.random.rand(int(1e8))

In [28]:
%%time
cfilt = signal.wiener(csig)

CPU times: user 31.9 s, sys: 11.9 s, total: 43.9 s
Wall time: 43.8 s


In [3]:
gsig = cp.random.randn(int(1e8))

In [4]:
%%time
gfilt = cusignal.wiener(gsig)

CPU times: user 586 ms, sys: 111 ms, total: 697 ms
Wall time: 700 ms


### Perform 1-D Hilbert Transform

In [5]:
csig = np.random.rand((int(1e8)))

In [6]:
%%time
chtrans = signal.hilbert(csig)

CPU times: user 12.5 s, sys: 3.54 s, total: 16.1 s
Wall time: 16 s


In [7]:
gsig = cp.random.rand(int(1e8))

In [8]:
%%time
ghtrans = cusignal.hilbert(gsig)

CPU times: user 45.5 ms, sys: 30.1 ms, total: 75.6 ms
Wall time: 73.7 ms


### Perform 2-D Hilbert Transform

In [9]:
csig = np.random.rand(int(1e4), int(1e4))

In [10]:
%%time
chtrans2d = signal.hilbert2(csig)

CPU times: user 11.6 s, sys: 4.37 s, total: 15.9 s
Wall time: 15.9 s


In [11]:
gsig = cp.random.rand(int(1e4), int(1e4))

In [12]:
%%time
ghtrans2d = cusignal.hilbert2(gsig)

CPU times: user 30.7 ms, sys: 12.9 ms, total: 43.6 ms
Wall time: 41.9 ms


### Perform 2-D Convolution and Correlation

In [13]:
csig = np.random.rand(int(1e4), int(1e4))
filt = np.random.rand(5,5)

In [14]:
%%time
grad = signal.convolve2d(csig, filt, boundary='symm', mode='same')

CPU times: user 8.73 s, sys: 366 ms, total: 9.1 s
Wall time: 9 s


In [15]:
%%time
grad = signal.correlate2d(csig, filt, boundary='symm', mode='same')

CPU times: user 8.72 s, sys: 402 ms, total: 9.12 s
Wall time: 9.04 s


In [16]:
gsig = cp.random.rand(int(1e4), int(1e4))
gfilt = cp.random.rand(5,5)

In [17]:
%%time
ggrad = cusignal.convolve2d(csig, filt, boundary='symm', mode='same')

CPU times: user 791 ms, sys: 1.1 s, total: 1.9 s
Wall time: 1.89 s


In [18]:
%%time
ggrad = cusignal.correlate2d(csig, filt, boundary='symm', mode='same')

CPU times: user 755 ms, sys: 635 ms, total: 1.39 s
Wall time: 1.39 s
